#**7th Week**

#**Задачи (Оба уровня)**

Тест состоит из задач по теме "Сеть". Вам предстоит решить ряд задач, демонстрирующих ваше понимание пройденной темы.  Вы можете проходить тест неограниченное количество раз, и в зачет идет ваш лучший результат. Удачи!


##**Сеть. Конвертация валют**

Необходимо разработать программу, которая позволяет пользователю конвертировать денежные средства из одной валюты в другую с помощью API.

Функция `get_exchange_rate(base_currency, target_currency)`:

Получает обозначения базовой (base_currency) и целевой (target_currency) валют, например - usd, EUR и т.д. Регистр обозначения валюты не должен влиять на работу программы. (См. документацию API - коды обозначений валют).

Возвращает текущий курс обмена между указанными валютами. Округлять и форматировать результат не нужно. (Используйте API)

Функция `convert_currency(base_currency, target_currency, amount)`:

Получает обозначения валют и кол-во базовой валюты для конвертации
Возвращает кол-во целевой валюты после конвертации. Округлить результат до двух знаков после запятой. (Используйте функцию get_exchange_rate для получения курса обмена валют)

Для получения актуальных курсов обмена валют используйте бесплатный API - https://github.com/fawazahmed0/exchange-api. (Читайте документацию - README репозитория)

###**Пример использования**
```
base_currency = "USD"  # Валюта, из которой нужно конвертировать
target_currency = "eur"  # Валюта, в которую нужно конвертировать
exchange_rate = get_exchange_rate(base_currency, target_currency)

print(f"Текущий курс {base_currency} к {target_currency}: {exchange_rate}")
# Вывод: Текущий курс USD к EUR: 0.9189

amount = 20 # Вводим кол-во базовой валюты, в данном примере - 20 USD
converted_amount = convert_currency(base_currency, target_currency, amount)

print(f"{amount} {base_currency} равно {converted_amount} {target_currency}")
# 20 USD равно 18.38 EUR
```


In [ ]:
def get_exchange_rate(base_currency, target_currency):
  base_currency = base_currency.lower()
  target_currency = target_currency.lower()
  url = f"https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies/{base_currency}.min.json"
  response = requests.get(url)
  if response.status_code == 200:
    data = response.json()
    if target_currency in data[base_currency]:
      return data[base_currency][target_currency]
    else:
      raise ValueError(f"Целевая валюта ({target_currency.upper()}) не найдена в данных API.")
  else:
    raise ValueError("Ошибка при запросе к API.")

def convert_currency(base_currency, target_currency, amount):
  exchange_rate = get_exchange_rate(base_currency, target_currency)
  if exchange_rate is None:
    raise ValueError("Невозможно получить курс обмена валют")
  converted_amount = round(amount * exchange_rate, 2)
  return converted_amount

##**Сеть**

Здесь описано некоторое API, в котором есть доступ к базе пользователей, постов, комментариев и т.д. Методы, которые мы будем использовать, описаны в разделе Resources. Примеры использования API (правда, на JavaScript'е) описаны на том же сайте по ссылке Guide. Вам нужно для каждого пользователя посчитать количество оставленных постов и количество оставленных комментариев. Всю информацию для этого нужно стягивать GET-запросами по API. Результат нужно отправить в ваше пространство в https://webhook.site в виде POST-запроса, содержащего JSON следующего формата:
```
  "statistics": [
    {
      "id": 1,
      "username": "lolkek",
      "email": "user1@mail.dot",
      "posts": 125,
      "comments": 1358
    },
    {
      "id": 2,
      "username": "cheburek",
      "email": "user2@mail.dot",
      "posts": 5,
      "comments": 12
    }
  ]
```

Задача - реализовать функцию `process_user_data()`, которая собирает информацию о пользователях из публичного API и отправляет статистику о каждом пользователе на заданный вебхук. Функция должна возвращать response объект ответа запроса -
`return requests.post(.....)`

Реализуйте функцию , которая возвращает результат в виде JSON следующего формата:

###**Пример использования**
```
# пример вызова функции
response = process_user_data()
```

In [ ]:
import requests

def process_user_data():
    # Получаем список пользователей
    users_list = requests.get('https://jsonplaceholder.typicode.com/users')
    users = users_list.json()

    # Получаем список постов
    posts_list = requests.get('https://jsonplaceholder.typicode.com/posts')
    posts = posts_list.json()

    # Получаем список комментариев
    comments_list = requests.get('https://jsonplaceholder.typicode.com/comments')
    comments = comments_list.json()

    # Составляем статистику по пользователям
    user_statistics = []

    # Для каждого пользователя собираем статистику
    for user in users:
        user_id = user['id']
        username = user['username']
        email = user['email']

        # Считаем количество постов для пользователя
        post_count = sum(1 for post in posts if post['userId'] == user_id)

        # Считаем количество комментариев для пользователя
        comment_count = sum(1 for comment in comments if any(post['userId'] == user_id for post in posts if post['id'] == comment['email']))

        # Добавляем статистику пользователя в список
        user_statistics.append({
            'id': user_id,
            'username': username,
            'email': email,
            'posts': post_count,
            'comments': comment_count
        })

    # Подготавливаем данные для отправки
    data = {
        'statistics': user_statistics
    }

    # URL для отправки данных (webhook URL)
    url = 'https://webhook.site/d779d06f-d643-4e82-8923-d5db5ad276a4'  # Ваш URL webhook

    # Отправляем POST запрос с данными
    response = requests.post(url, json=data)

    return response

